In [1]:
import pandas as pd
from pathlib import Path

In [2]:
tez_path = '../input/tez-lib/'
effnet_path = '../input/efficientnet-pytorch/'
import sys
sys.path.append(tez_path)
sys.path.append(effnet_path)


In [3]:
import os
import albumentations
import pandas as pd
import numpy as np

import tez
from tez.datasets import ImageDataset

import torch
import torch.nn as nn
from torch.nn import functional as F

from efficientnet_pytorch import EfficientNet


ModuleNotFoundError: No module named 'tez'

In [4]:
INPUT_PATH = "../input/ranzcr-clip-catheter-line-classification/"
IMAGE_PATH = "../input/ranzcr-clip-catheter-line-classification/test/"
MODEL_PATH = "../input/ranzcr-effnet5/"
IMAGE_SIZE = 512


In [5]:
df = pd.read_csv(os.path.join(INPUT_PATH, "sample_submission.csv"))


In [6]:
class RanzcrModel(tez.Model):
    def __init__(self):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b5")

        self.effnet._conv_stem.in_channels = 1
        weight = self.effnet._conv_stem.weight.mean(1, keepdim=True)
        self.effnet._conv_stem.weight = torch.nn.Parameter(weight)

        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(2048, 11)

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs, None, {}


NameError: name 'tez' is not defined

In [7]:
test_aug = albumentations.Compose(
    [
        albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        albumentations.Normalize(
            mean=[0.485],
            std=[0.229],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)


In [8]:
test_image_paths = [
    os.path.join(IMAGE_PATH, x + ".jpg") 
    for x in df.StudyInstanceUID.values
]


In [9]:
test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=[0]*len(test_image_paths),
    augmentations=test_aug,
    grayscale=True,
)


NameError: name 'ImageDataset' is not defined

In [10]:
model = RanzcrModel()
model.load(os.path.join(MODEL_PATH, "effnet5_fold_0.bin"))


NameError: name 'RanzcrModel' is not defined

In [11]:
preds = model.predict(
    test_dataset, batch_size=32, n_jobs=-1, device="cuda"
)
temp_preds = None
for p in preds:
    if temp_preds is None:
        temp_preds = p
    else:
        temp_preds = np.vstack((temp_preds, p))


NameError: name 'model' is not defined

In [12]:
target_cols = df.columns[1:]

for i in range(temp_preds.shape[1]):
    df.loc[:, target_cols[i]] = temp_preds[:, i]


NameError: name 'temp_preds' is not defined

In [13]:
df.to_csv('submission.csv', index=False)
df.head()


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline
0,1.2.826.0.1.3680043.8.498.25512976433640891933...,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.24449897997512078380...,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.38485493636649999035...,0,0,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.8.498.11901773728604504629...,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.11327663178389439022...,0,0,0,0,0,0,0,0,0
